In [1]:
from hathor import Hathor
import sys
import os
hathor_root = os.path.dirname(os.getcwd())
sys.path.insert(0, hathor_root)
from inference_auth_token import get_access_token
access_token = get_access_token()
config_list = [{
    "model": "openai/gpt-oss-120b",  # or whatever model name your endpoint expects
    "api_key": access_token,
    "base_url": "https://inference-api.alcf.anl.gov/resource_server/sophia/vllm/v1",
    "api_type": "openai",  # Important: tells AutoGen to use OpenAI format
}]
hathor = Hathor(config_list=config_list)
hathor.run()

/opt/miniconda3/envs/globus_env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


UserProxy (to chat_manager):

Please create 20 hypotheses about galaxy formation.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.


PermissionDeniedError: Error code: 403 - {'detail': "Error: Permission denied from internal policies. This is likely due to a high-assurance timeout. Please logout by visiting https://app.globus.org/logout, and re-authenticate with the following command: 'python3 inference_auth_token.py authenticate --force'. Make sure you authenticate with an authorized identity provider: anl.gov, alcf.anl.gov, or providers with approved projects."}